3)	На основе тестовых таблиц payment.csv и users.csv постройте отчёт в системе аналитики (powerBI), содержащий графики отражающие динамику следующих показателей в зависимости от количества дней прошедших с даты регистрации пользователя и принимает следующие значения 3,7,14,28 день

- % платящих
- ARPU 
- ARPPU 
- Retention 


In [1]:
# Ипортируем необходимые библиотеки
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px

In [2]:
# создаем df из csv-файлов
payment = pd.read_csv('payment.csv')
payment.columns =['id','transaction_time','amount']

users = pd.read_csv('users.csv')

In [3]:
#меняем тип данных даты
payment['transaction_time'] = pd.to_datetime(payment['transaction_time'])
payment['transaction_time'] = payment['transaction_time'].dt.to_period('D')

users['create'] = pd.to_datetime(users['create'])
users['create'] = users['create'].dt.to_period('D')

In [4]:
# Изменяем тип данных в каждой строчке столбца ретеншен, удалю все NaN
users_up = users.dropna().reset_index(drop=True)
for i in range(len(users_up['retention'])):
    users_up['retention'][i] = [int(x) for x in users_up['retention'][i].split(',')]
users_up


C:\Users\4dani\AppData\Local\Temp/ipykernel_936/4203628238.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_up['retention'][i] = [int(x) for x in users_up['retention'][i].split(',')]


,id,create,rev,retention
0,301081861400,2022-05-25,0,[0]
1,549655335280,2022-05-25,0,"[31, 32, 33]"
2,600057195030,2022-05-25,0,"[6, 8, 12]"
3,581289985070,2022-05-25,0,"[3, 22]"
4,508929760860,2022-05-25,0,"[1, 3, 4]"
...,...,...,...,...
94,377614566160,2022-05-25,0,"[2, 3, 5, 6, 7, 8]"
95,539012951580,2022-05-25,0,"[1, 2, 3, 4, 5, 11]"
96,338650686400,2022-05-25,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
97,326415233420,2022-05-25,0,"[3, 4, 20, 26, 30, 31, 33]"


In [5]:
# Соединяем таблицы
df = payment.merge(users_up, on='id', how='inner')
display(df.head())
display(df.tail())

,id,transaction_time,amount,create,rev,retention
0,341084584630,2022-05-27,100,2022-05-25,0,"[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ..."
1,341084584630,2022-05-30,70,2022-05-25,0,"[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ..."
2,341084584630,2022-06-05,200,2022-05-25,0,"[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ..."
3,555298420800,2022-05-26,80,2022-05-25,0,[4]
4,559803669980,2022-05-26,100,2022-05-25,0,"[5, 6, 7, 28, 31, 33]"


,id,transaction_time,amount,create,rev,retention
19,561348234960,2022-06-05,250,2022-05-25,0,"[1, 2, 3, 4, 6, 7, 13, 14, 18, 20, 21, 23, 30,..."
20,561348234960,2022-06-07,100,2022-05-25,0,"[1, 2, 3, 4, 6, 7, 13, 14, 18, 20, 21, 23, 30,..."
21,553025346770,2022-06-10,50,2022-05-25,0,"[1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 14, 15, 16, 1..."
22,553025346770,2022-06-12,10,2022-05-25,0,"[1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 14, 15, 16, 1..."
23,136231146450,2022-05-25,40,2022-05-25,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


## Стоит заметить что мы потеряли 1 строчку из таблицы payment, это произошло потому что у этого ID не было даты регистрации, я бы отправил баг-репорт

In [6]:
# Вычисляем сколько прошло дней с регистрации при совершении платежа
df['day_when_pay'] = df['transaction_time'] - df['create']
df['day_when_pay'] = df['day_when_pay'] / np.timedelta64(1, 'D')
df['day_when_pay'] = df['day_when_pay'].astype('int')
display(df.head())
display(df.tail())

,id,transaction_time,amount,create,rev,retention,day_when_pay
0,341084584630,2022-05-27,100,2022-05-25,0,"[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...",2
1,341084584630,2022-05-30,70,2022-05-25,0,"[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...",5
2,341084584630,2022-06-05,200,2022-05-25,0,"[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...",11
3,555298420800,2022-05-26,80,2022-05-25,0,[4],1
4,559803669980,2022-05-26,100,2022-05-25,0,"[5, 6, 7, 28, 31, 33]",1


,id,transaction_time,amount,create,rev,retention,day_when_pay
19,561348234960,2022-06-05,250,2022-05-25,0,"[1, 2, 3, 4, 6, 7, 13, 14, 18, 20, 21, 23, 30,...",11
20,561348234960,2022-06-07,100,2022-05-25,0,"[1, 2, 3, 4, 6, 7, 13, 14, 18, 20, 21, 23, 30,...",13
21,553025346770,2022-06-10,50,2022-05-25,0,"[1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 14, 15, 16, 1...",16
22,553025346770,2022-06-12,10,2022-05-25,0,"[1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 14, 15, 16, 1...",18
23,136231146450,2022-05-25,40,2022-05-25,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0


In [7]:
# Создам итоговую таблицу, к которой буду прилеплять все вычисления
a = []
for i in range(29):
    a.append(i)
final_table = pd.DataFrame(a)
final_table.columns = ['lifetime']
final_table['count_payers'] = final_table['lifetime']

In [8]:
# Вычисляем количество платящих
for i in range(final_table['lifetime'].min(), final_table['lifetime'].max()+1):
    final_table['count_payers'][i] = df[df['day_when_pay']<=i]['id'].nunique()
final_table.head()

,lifetime,count_payers
0,0,2
1,1,4
2,2,5
3,3,5
4,4,5


In [9]:
# Вычисляем процент платящих
final_table['percent_payers'] = round(final_table['count_payers'] / users['id'].nunique(),2)
final_table.head()

,lifetime,count_payers,percent_payers
0,0,2,0.01
1,1,4,0.02
2,2,5,0.03
3,3,5,0.03
4,4,5,0.03


In [10]:
# Вычисляем ARPU и ARPPU
final_table['ARPU'] = final_table['lifetime']
for i in range(final_table['lifetime'].min(), final_table['lifetime'].max()+1):
    final_table['ARPU'][i] = df[df['day_when_pay']<=i]['amount'].sum()
final_table['ARPPU'] = round(final_table['ARPU']/df['id'].nunique(),2)
final_table['ARPU'] = round(final_table['ARPU']/users['id'].nunique(),2)
final_table.head()

C:\Users\4dani\AppData\Local\Temp/ipykernel_936/2814347810.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_table['ARPU'][i] = df[df['day_when_pay']<=i]['amount'].sum()


,lifetime,count_payers,percent_payers,ARPU,ARPPU
0,0,2,0.01,0.80,15.56
1,1,4,0.02,1.83,35.56
2,2,5,0.03,2.40,46.67
3,3,5,0.03,2.40,46.67
4,4,5,0.03,2.40,46.67


In [11]:
# Вычисляем classic Retention
final_table['Retention'] = final_table['lifetime']
for i in range(final_table['lifetime'].min(), final_table['lifetime'].max()+1):
    final_table['Retention'][i] = users_up[users_up['retention'].map(lambda v: i in v)]['id'].nunique()
final_table


C:\Users\4dani\AppData\Local\Temp/ipykernel_936/2071072624.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_table['Retention'][i] = users_up[users_up['retention'].map(lambda v: i in v)]['id'].nunique()


,lifetime,count_payers,percent_payers,ARPU,ARPPU,Retention
0,0,2,0.01,0.80,15.56,19
1,1,4,0.02,1.83,35.56,33
2,2,5,0.03,2.40,46.67,30
3,3,5,0.03,2.40,46.67,34
4,4,5,0.03,2.40,46.67,36
5,5,5,0.03,3.09,60.00,27
6,6,5,0.03,3.09,60.00,30
7,7,6,0.03,8.80,171.11,29
8,8,7,0.04,9.20,178.89,27
9,9,7,0.04,9.83,191.11,20


In [13]:
# загружаю полученную таблицу в файл, чтобы продолжить работу в Power BI
final_table.to_csv('final_table.csv')